## Importing pandas

In [1]:
import pandas as pd

## Load data

In [2]:
#load data
facilities = pd.read_csv('facilities.csv')
facilities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35574 entries, 0 to 35573
Data columns (total 47 columns):
uid            35574 non-null int64
hash           35574 non-null object
idold          189 non-null object
idagency       24599 non-null object
facname        35567 non-null object
addressnum     33130 non-null object
streetname     33130 non-null object
address        35040 non-null object
city           35542 non-null object
boro           35574 non-null object
borocode       35574 non-null int64
zipcode        35426 non-null float64
latitude       35574 non-null float64
longitude      35574 non-null float64
xcoord         35574 non-null float64
ycoord         35574 non-null float64
bin            31341 non-null object
bbl            34246 non-null object
commboard      35385 non-null float64
council        35574 non-null int64
censtract      35574 non-null int64
nta            35385 non-null object
facdomain      35574 non-null object
facgroup       35574 non-null object
fac

In [3]:
#View first 5 rows
facilities.head()

,uid,hash,idold,idagency,facname,addressnum,streetname,address,city,boro,...,overagency,overabbrev,datasource,dataname,datalink,datadate,pgtable,uid_merged,hash_merged,geom
0,33492,a789c225b1ed04091a7eb14a93c8253c,NaN,NYCDCA: 1430208-DCA,Ud Parking Corp.,761,East 168 Street,761 East 168 Street,Bronx,Bronx,...,NYC Department of Consumer Affairs,NYCDCA,NYCDCA,NYCDCA: Legally Operating Businesses,NYCDCA: https://data.cityofnewyork.us/Business...,NYCDCA: 2017-01-20,dca_facilities_operatingbusinesses,NaN,NaN,0101000020E6100000FD37F6A1A97952C008247B61306A...
1,11451,77ec8a8ffdbe776aeeed1c37d6de9f34,NaN,NYCDOE: K274,P.S. 274 Kosciusko,800,Bushwick Avenue,800 Bushwick Avenue,Brooklyn,Brooklyn,...,NYC Department of Education,NYCDOE,NaN,:,:,:,doe_facilities_universalprek;dcas_facilities_colp,NaN,NaN,0101000020E6100000934B4BCC637B52C0A8AB7463E258...
2,40506,64e3f413da65472ca5e39d7b31c02c95,NaN,NYSOMH: 872099-7796-001,Brooklyn Mental Health Court,320,Jay Street,320 Jay Street,Brooklyn,Brooklyn,...,NYS Office of Mental Health,NYSOMH,NYSOMH,NYSOMH: Local Mental Health Programs,NYSOMH: https://data.ny.gov/Human-Services/Loc...,NYSOMH: 2016-10-24,nysomh_facilities_mentalhealth,NaN,NaN,0101000020E61000000E172F93367F52C0DB735816E958...
3,16153,fef53b0de14124f1e69037d4a04add59,NaN,NYCDOE: K204,P.S. 204 - K,8101,15 Avenue,8101 15 Avenue,Brooklyn,Brooklyn,...,NYC Department of Education,NYCDOE,NaN,:,:,:,dcas_facilities_colp;doe_facilities_schoolsblu...,16154,699f2d2b70a5bb8e13663b04eeb796bb,0101000020E610000028EAC645688052C097A69F298D4E...
4,11665,04f0545585eaaef29a3e505d18b3014a,NaN,NaN,Beach Channel Drive,NaN,NaN,Beach 108 Street,Rockaway Park,Queens,...,NYC Department of Environmental Protection,NYCDEP,NYCDCAS,NYCDCAS: City Owned and Leased Properties,NYCDCAS: http://www1.nyc.gov/site/planning/dat...,NYCDCAS: 2016-10-20,dcas_facilities_colp,5357,160c18a9e48bf3879daddf618e03eba2,0101000020E6100000FA59D0D72E7552C07C88A81CC74A...


## Eliminate duplicate rows that exhibit a 100% match of name and address of facilities

In [4]:
#Filter out duplicate rows
cleaned = facilities.drop_duplicates(subset=['facname','address','idagency'])
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33571 entries, 0 to 35573
Data columns (total 47 columns):
uid            33571 non-null int64
hash           33571 non-null object
idold          182 non-null object
idagency       23634 non-null object
facname        33567 non-null object
addressnum     31275 non-null object
streetname     31275 non-null object
address        33046 non-null object
city           33545 non-null object
boro           33571 non-null object
borocode       33571 non-null int64
zipcode        33436 non-null float64
latitude       33571 non-null float64
longitude      33571 non-null float64
xcoord         33571 non-null float64
ycoord         33571 non-null float64
bin            29522 non-null object
bbl            32255 non-null object
commboard      33404 non-null float64
council        33571 non-null int64
censtract      33571 non-null int64
nta            33404 non-null object
facdomain      33571 non-null object
facgroup       33571 non-null object
fac

## Eliminating rows that have a null values for name and address

In [5]:
#Eliminating null values in name
cleaned = cleaned.dropna(subset=['facname'])
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33567 entries, 0 to 35573
Data columns (total 47 columns):
uid            33567 non-null int64
hash           33567 non-null object
idold          182 non-null object
idagency       23634 non-null object
facname        33567 non-null object
addressnum     31273 non-null object
streetname     31273 non-null object
address        33043 non-null object
city           33541 non-null object
boro           33567 non-null object
borocode       33567 non-null int64
zipcode        33432 non-null float64
latitude       33567 non-null float64
longitude      33567 non-null float64
xcoord         33567 non-null float64
ycoord         33567 non-null float64
bin            29521 non-null object
bbl            32253 non-null object
commboard      33400 non-null float64
council        33567 non-null int64
censtract      33567 non-null int64
nta            33400 non-null object
facdomain      33567 non-null object
facgroup       33567 non-null object
fac

In [6]:
#Eliminating null values in name
cleaned = cleaned.dropna(subset=['address'])
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33043 entries, 0 to 35573
Data columns (total 47 columns):
uid            33043 non-null int64
hash           33043 non-null object
idold          182 non-null object
idagency       23171 non-null object
facname        33043 non-null object
addressnum     31273 non-null object
streetname     31273 non-null object
address        33043 non-null object
city           33027 non-null object
boro           33043 non-null object
borocode       33043 non-null int64
zipcode        32993 non-null float64
latitude       33043 non-null float64
longitude      33043 non-null float64
xcoord         33043 non-null float64
ycoord         33043 non-null float64
bin            29513 non-null object
bbl            32225 non-null object
commboard      32961 non-null float64
council        33043 non-null int64
censtract      33043 non-null int64
nta            32961 non-null object
facdomain      33043 non-null object
facgroup       33043 non-null object
fac

## Testing the preliminary cleaning for Polly Dodge Early Learning Center

In [7]:
#Verifying for Polly Dodge Early Learning Center
temp = cleaned[cleaned['bin']=='1086807']
temp[['facname','facdomain','facgroup','facsubgrp','factype','zipcode']]

,facname,facdomain,facgroup,facsubgrp,factype,zipcode
5815,YWCA - NYC Polly Dodge ELC,"Education, Child Welfare, and Youth",Child Care and Pre-Kindergarten,Dual Child Care and Universal Pre-K,Early Education Program,10019.0
6233,Polly Dodge Early Learning Center,"Education, Child Welfare, and Youth",Child Care and Pre-Kindergarten,Child Care,Early Learn NYC: New York City's Early Care an...,10019.0
12156,Young Women's Christian Association Of The Cit...,"Education, Child Welfare, and Youth",Child Care and Pre-Kindergarten,Child Care,Group Day Care - Preschool,10019.0


## Additional methods of cleaning facility names - 'Inc' 

In [8]:
#checking for names that contain 'Inc'
with_inc = cleaned[cleaned['facname'].str.contains("Inc")]
with_inc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4698 entries, 5 to 35569
Data columns (total 47 columns):
uid            4698 non-null int64
hash           4698 non-null object
idold          0 non-null object
idagency       2329 non-null object
facname        4698 non-null object
addressnum     4665 non-null object
streetname     4665 non-null object
address        4698 non-null object
city           4698 non-null object
boro           4698 non-null object
borocode       4698 non-null int64
zipcode        4697 non-null float64
latitude       4698 non-null float64
longitude      4698 non-null float64
xcoord         4698 non-null float64
ycoord         4698 non-null float64
bin            4479 non-null object
bbl            4564 non-null object
commboard      4698 non-null float64
council        4698 non-null int64
censtract      4698 non-null int64
nta            4698 non-null object
facdomain      4698 non-null object
facgroup       4698 non-null object
facsubgrp      4698 non-null 

In [9]:
with_inc.head()

,uid,hash,idold,idagency,facname,addressnum,streetname,address,city,boro,...,overagency,overabbrev,datasource,dataname,datalink,datadate,pgtable,uid_merged,hash_merged,geom
5,43363,8a009e4773dfb5e793a0aed9eaa8789f,NaN,NaN,Adult Resources Center Inc. (Dba Arc),1145,East 55 Street,1145 East 55 Street,Brooklyn,Brooklyn,...,NYS Office for People With Developmental Disab...,NYSOPWDD,NYSOPWDD,NYSOPWDD: Directory of Developmental Disabilit...,NYSOPWDD: https://data.ny.gov/Human-Services/D...,NYSOPWDD: 2015-12-22,nysopwdd_facilities_providers,NaN,NaN,0101000020E610000014EA47710F7B52C0CED7B54C8350...
6,29209,b5bc0b9e2682490ccc139ec8b842b3ee,NaN,NaN,New Mustang Auto Parts & Recycling Inc,127-54,Willets Point Boulevard,127-54 Willets Point Boulevard,Corona,Queens,...,NYS Department of Environmental Conservation,NYSDEC,NYSDEC,NYSDEC: Solid Waste Management Facilities,NYSDEC: https://data.ny.gov/Energy-Environment...,NYSDEC: 2017-01-09,nysdec_facilities_solidwaste,NaN,NaN,0101000020E6100000DF4E79ABD07552C0CFA986E14961...
21,35691,3b7af1d2c65e593f96307acc57e16d0b,NaN,NaN,"Amas Musical Theatre, Inc.",257,West 52 Street,257 West 52 Street,New York,Manhattan,...,NYC Department of Cultural Affairs,NYCDCLA,NYCDCLA,NYCDCLA: DCLA Cultural Organizations,NYCDCLA: https://data.cityofnewyork.us/Recreat...,NYCDCLA: 2016-03-22,dcla_facilities_culturalinstitutions,NaN,NaN,0101000020E610000008012006FD7E52C09715F234B661...
30,36905,dba578a9be2b267315543d98dc54043a,NaN,NYCDCA: 0369956-DCA,State-Pearl Garage Inc.,1,Battery Park Plaza,1 Battery Park Plaza,New York,Manhattan,...,NYC Department of Consumer Affairs,NYCDCA,NYCDCA,NYCDCA: Legally Operating Businesses,NYCDCA: https://data.cityofnewyork.us/Business...,NYCDCA: 2017-01-20,dca_facilities_operatingbusinesses,NaN,NaN,0101000020E6100000E46533A1E18052C03D950219085A...
31,40177,5507301cfc715b8a1a65ce5473cc37fa,NaN,NYSOASAS: 7371,"The Bridge, Inc. OP",248,West 108 Street,248 West 108 Street,New York,Manhattan,...,NYS Office of Alcoholism and Substance Abuse S...,NYSOASAS,NYSOASAS,NYSOASAS: List of NYC Programs,NYSOASAS:,NYSOASAS: 2017-01-03,nysoasas_facilities_programs,NaN,NaN,0101000020E6100000CE257A32E37D52C049D14DFFB566...


## Test the cleaning with example

In [10]:
bx = cleaned[cleaned['facname'].str.contains("Bronx City")]
bx[['facname','facdomain','facgroup','facsubgrp','factype','bbl', 'idagency','address','boro']]

,facname,facdomain,facgroup,facsubgrp,factype,bbl,idagency,address,boro
12351,Bronx City Recycling Inc,Core Infrastructure and Transportation,Solid Waste,Solid Waste Transfer and Carting,Trade Waste Carter Site,2027770417,NaN,1390 Viele Avenue,Bronx
13578,Bronx City Recycling (1390 Viele Ave),Core Infrastructure and Transportation,Solid Waste,Solid Waste Processing,Construction and Demolition Processing,2027770417,NYSDEC: ?03W88,1390 Viele Avenue,Bronx


In [11]:
#removing Inc

sep = 'Inc'
head,sep,tail = 'Bronx City Recycling Inc'.partition('Inc')
head+tail

'Bronx City Recycling '

## The dataframe 'cleaned' can be used as input for the 'similarity_matrix' function that follows:

In [ ]:
zip_10019 = 